In [ ]:
# 리눅스 파이썬 환경 구축
 # vscode 설치(코드로 뭐하던데 편하게 클릭 ㄱㄱ)
 # vscode 확장자로 python과 python extension pack 설치
 # bash 셀( 그래도 오류나면 커널 global env로 해보기)
   # sudo apt install python3-pip
   # python3-m pip config set global.break-system-packages true
   # /usr/bin/python3 -m pip install ipykernel -U --user --force-reinstall
   # 하고 vscode 재실행

In [1]:
print('a')

a


In [2]:
a = 1
b = 2
a + b

3

In [3]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
import mysql.connector

# API 키 입력
API_KEY = "A73F3BA4D1A04658BAAFE34134E80CE9E0FE4655"

# API 엔드포인트
BASE_URL = {
    'kospi': 'http://data-dbg.krx.co.kr/svc/apis/sto/stk_bydd_trd',
    'kosdaq': 'http://data-dbg.krx.co.kr/svc/apis/sto/ksq_bydd_trd',
    'konex': 'http://data-dbg.krx.co.kr/svc/apis/sto/knx_bydd_trd'
}

# MySQL 연결 설정
db_config = {
    'host': 'localhost',  # MySQL 서버 주소
    'user': 'pdc',        # MySQL 사용자 이름
    'password': '1234',  # MySQL 비밀번호
    'database': 'CJD'     # 데이터베이스 이름
}

# 데이터 요청 함수
def fetch_data(market, date, max_retries=3, retry_delay=2):
    url = BASE_URL[market]
    headers = {'AUTH_KEY': API_KEY}
    params = {'basDd': date}

    for attempt in range(max_retries):
        try:
            response = requests.get(url, headers=headers, params=params)
            
            if response.status_code == 200:
                data = response.json()
                return data.get('OutBlock_1', [])  # 필요한 데이터 블록 반환
            
            else:
                print(f"Error {response.status_code} for {market} on {date}. Retrying...")
        
        except requests.exceptions.RequestException as e:
            print(f"Request error for {market} on {date}: {e}. Retrying...")

        # 딜레이 추가
        time.sleep(retry_delay)

    # 최대 재시도 후에도 실패한 경우
    print(f"Failed to fetch data for {market} on {date} after {max_retries} retries.")
    return []

# 날짜 생성 함수
def generate_dates(start_date, end_date):
    dates = []
    current_date = start_date
    while current_date <= end_date:
        dates.append(current_date.strftime('%Y%m%d'))
        current_date += timedelta(days=1)
    return dates

# 데이터베이스에 데이터 저장 함수
def save_to_mysql(data, table_name):
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    # 테이블 생성 (없는 경우)
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        basDd VARCHAR(8),
        isuCd VARCHAR(20),
        isuNm VARCHAR(100),
        market VARCHAR(10),
        tddOpnprc FLOAT,
        tddHgprc FLOAT,
        tddLwprc FLOAT,
        tddClsprc FLOAT,
        accTrdvol BIGINT,
        accTrdval BIGINT
    );
    """
    cursor.execute(create_table_query)
    conn.commit()

    # 데이터 삽입
    insert_query = f"""
    INSERT INTO {table_name} 
    (basDd, isuCd, isuNm, market, tddOpnprc, tddHgprc, tddLwprc, tddClsprc, accTrdvol, accTrdval)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """
    for record in data:
        cursor.execute(insert_query, (
            record.get('basDd'), record.get('isuCd'), record.get('isuNm'), record.get('market'),
            record.get('tddOpnprc'), record.get('tddHgprc'), record.get('tddLwprc'), record.get('tddClsprc'),
            record.get('accTrdvol'), record.get('accTrdval')
        ))
    conn.commit()
    cursor.close()
    conn.close()
    print(f"Data saved to MySQL table {table_name}")

# 데이터 수집 및 저장
def collect_and_save_data_to_mysql(start_date, end_date, table_name):
    dates = generate_dates(start_date, end_date)
    markets = ['kospi', 'kosdaq', 'konex']
    
    for date in dates:
        for market in markets:
            print(f"Fetching data for {market.upper()} on {date}...")
            data = fetch_data(market, date)
            if data:
                for record in data:
                    record['market'] = market  # 시장 정보를 추가
                
                # MySQL에 저장
                save_to_mysql(data, table_name)
            
            # 요청 사이 딜레이
            time.sleep(1)

# 실행
if __name__ == "__main__":
    start_date = datetime.strptime("20100816", "%Y%m%d")
    end_date = datetime.today()
    table_name = "krx_market_data"
    
    collect_and_save_data_to_mysql(start_date, end_date, table_name)

Fetching data for KOSPI on 20100816...
Data saved to MySQL table krx_market_data
Fetching data for KOSDAQ on 20100816...
Request error for kosdaq on 20100816: HTTPConnectionPool(host='data-dbg.krx.co.kr', port=80): Max retries exceeded with url: /svc/apis/sto/ksq_bydd_trd?basDd=20100816 (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x7a4c7c177680>: Failed to resolve 'data-dbg.krx.co.kr' ([Errno -3] Temporary failure in name resolution)")). Retrying...
Request error for kosdaq on 20100816: HTTPConnectionPool(host='data-dbg.krx.co.kr', port=80): Max retries exceeded with url: /svc/apis/sto/ksq_bydd_trd?basDd=20100816 (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x7a4c69f101a0>: Failed to resolve 'data-dbg.krx.co.kr' ([Errno -3] Temporary failure in name resolution)")). Retrying...
Request error for kosdaq on 20100816: HTTPConnectionPool(host='data-dbg.krx.co.kr', port=80): Max retries exceeded with url: /svc/apis/sto/ksq_bydd

KeyboardInterrupt: 

In [2]:
import os
import subprocess
import requests
import mysql.connector
from datetime import datetime, timedelta

# API 키 입력
API_KEY = "A73F3BA4D1A04658BAAFE34134E80CE9E0FE4655"

# API 엔드포인트
BASE_URL = {
    'kospi': 'http://data-dbg.krx.co.kr/svc/apis/sto/stk_bydd_trd',
    'kosdaq': 'http://data-dbg.krx.co.kr/svc/apis/sto/ksq_bydd_trd',
    'konex': 'http://data-dbg.krx.co.kr/svc/apis/sto/knx_bydd_trd'
}

# MySQL 연결 설정
db_config = {
    'host': 'localhost',
    'user': 'pdc',
    'password': '1234',
    'database': 'CJD'
}

# 상태 파일 경로
state_file = "/home/one/DC/progress_first.txt"

def read_last_date():
    """상태 파일에서 마지막으로 처리한 날짜를 읽습니다."""
    if os.path.exists(state_file):
        with open(state_file, "r") as file:
            return datetime.strptime(file.read().strip(), "%Y%m%d")
    return datetime.strptime("20100816", "%Y%m%d")  # 기본 시작 날짜

def write_last_date(date):
    """마지막으로 처리한 날짜를 상태 파일에 저장합니다."""
    with open(state_file, "w") as file:
        file.write(date.strftime("%Y%m%d"))

def fetch_data(market, date):
    """API에서 데이터를 가져옵니다."""
    url = BASE_URL[market]
    headers = {'AUTH_KEY': API_KEY}
    params = {'basDd': date}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json().get('OutBlock_1', [])
    else:
        print(f"Error fetching data for {market} on {date}: {response.status_code}")
        return []

def save_to_mysql(data, table_name):
    """MySQL 데이터베이스에 데이터를 저장합니다."""
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        basDd TEXT,
        isuCd TEXT,
        isuNm TEXT,
        market TEXT,
        tddOpnprc TEXT,
        tddHgprc TEXT,
        tddLwprc TEXT,
        tddClsprc TEXT,
        accTrdvol TEXT,
        accTrdval TEXT
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    insert_query = f"""
    INSERT INTO {table_name} (
        basDd, isuCd, isuNm, market, tddOpnprc, tddHgprc, tddLwprc,
        tddClsprc, accTrdvol, accTrdval
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """
    for record in data:
        cursor.execute(insert_query, (
            record.get('basDd'), record.get('isuCd'), record.get('isuNm'), record.get('market'),
            record.get('tddOpnprc'), record.get('tddHgprc'), record.get('tddLwprc'), record.get('tddClsprc'),
            record.get('accTrdvol'), record.get('accTrdval')
        ))
    conn.commit()
    cursor.close()
    conn.close()
    print(f"Data saved to MySQL table {table_name}")

def collect_data(current_date, chunk_size, end_date, table_name):
    """데이터를 수집하고 즉시 데이터베이스에 저장합니다."""
    next_end_date = min(current_date + chunk_size, end_date)
    dates = [current_date + timedelta(days=i) for i in range((next_end_date - current_date).days + 1)]
    
    for date in dates:
        for market in ['kospi', 'kosdaq', 'konex']:
            print(f"Fetching data for {market.upper()} on {date.strftime('%Y%m%d')}...")
            data = fetch_data(market, date.strftime('%Y%m%d'))
            if data:
                for record in data:
                    record['market'] = market
                save_to_mysql(data, table_name)
    return next_end_date

def switch_to_second_file():
    """크론 작업에서 첫 번째 파일을 비활성화하고 두 번째 파일을 활성화합니다."""
    crontab = subprocess.run(["crontab", "-l"], capture_output=True, text=True).stdout
    new_crontab = []
    for line in crontab.splitlines():
        if "first_file.py" not in line:
            new_crontab.append(line)
    new_crontab.append("0 0 * * * /usr/bin/python3 /home/one/DC/second_file.py")
    new_crontab_text = "\n".join(new_crontab)
    subprocess.run(["crontab"], input=new_crontab_text, text=True)
    print("Switched crontab to second_file.py.")

def main():
    current_date = read_last_date()
    end_date = datetime.today() - timedelta(days=1)
    chunk_size = timedelta(days=5 * 365)
    table_name = "krx_market_data"

    if current_date >= end_date:
        print("All data processed. Switching to second file.")
        switch_to_second_file()
        return

    next_end_date = collect_data(current_date, chunk_size, end_date, table_name)
    write_last_date(next_end_date + timedelta(days=1))
    print(f"Progress updated. Next start date: {next_end_date + timedelta(days=1).strftime('%Y-%m-%d')}")

def test_fetch_sample_data():
    """API 데이터를 샘플로 확인합니다."""
    test_date = "20231120"  # YYYYMMDD 형식
    test_market = "kospi"  # kospi, kosdaq, konex 중 하나

    print(f"Testing data fetch for {test_market.upper()} on {test_date}...")
    data = fetch_data(test_market, test_date)
    if data:
        print(f"Fetched {len(data)} records for {test_market.upper()} on {test_date}.")
        print("Sample record:", data[0])
    else:
        print(f"No data fetched for {test_market.upper()} on {test_date}.")

if __name__ == "__main__":
    print("Running full system test...")
    test_fetch_sample_data()
    main()


Running full system test...
Testing data fetch for KOSPI on 20231120...
Fetched 952 records for KOSPI on 20231120.
Sample record: {'BAS_DD': '20231120', 'ISU_CD': '095570', 'ISU_NM': 'AJ네트웍스', 'MKT_NM': 'KOSPI', 'SECT_TP_NM': '-', 'TDD_CLSPRC': '4165', 'CMPPREVDD_PRC': '15', 'FLUC_RT': '0.36', 'TDD_OPNPRC': '4120', 'TDD_HGPRC': '4205', 'TDD_LWPRC': '4120', 'ACC_TRDVOL': '113979', 'ACC_TRDVAL': '474497370', 'MKTCAP': '188477741235', 'LIST_SHRS': '45252759'}
Fetching data for KOSPI on 20100816...
Data saved to MySQL table krx_market_data
Fetching data for KOSDAQ on 20100816...
Data saved to MySQL table krx_market_data
Fetching data for KONEX on 20100816...
Fetching data for KOSPI on 20100817...
Data saved to MySQL table krx_market_data
Fetching data for KOSDAQ on 20100817...
Data saved to MySQL table krx_market_data
Fetching data for KONEX on 20100817...
Fetching data for KOSPI on 20100818...
Data saved to MySQL table krx_market_data
Fetching data for KOSDAQ on 20100818...
Data saved to 

KeyboardInterrupt: 

In [15]:
import requests

# 설정 정보
API_KEY = "A73F3BA4D1A04658BAAFE34134E80CE9E0FE4655"
BASE_URL = {
    'kospi': 'http://data-dbg.krx.co.kr/svc/apis/sto/stk_bydd_trd',
    'kosdaq': 'http://data-dbg.krx.co.kr/svc/apis/sto/ksq_bydd_trd',
    'konex': 'http://data-dbg.krx.co.kr/svc/apis/sto/knx_bydd_trd'
}

def fetch_data(market, date):
    """API에서 데이터를 가져옵니다."""
    url = BASE_URL[market]
    headers = {'AUTH_KEY': API_KEY}
    params = {'basDd': date}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json().get('OutBlock_1', [])
        print(f"Fetched data for {market.upper()} on {date}:")
        for record in data:
            print(record)
        return data
    else:
        print(f"Error fetching data for {market.upper()} on {date}: {response.status_code}")
        return []

def main():
    date = input("Enter the date (YYYYMMDD): ")
    for market in ['kospi', 'kosdaq', 'konex']:
        print(f"\nFetching data for {market.upper()} on {date}...")
        fetch_data(market, date)

if __name__ == "__main__":
    main()


Fetching data for KOSPI on 20241120...
Fetched data for KOSPI on 20241120:
{'BAS_DD': '20241120', 'ISU_CD': '095570', 'ISU_NM': 'AJ네트웍스', 'MKT_NM': 'KOSPI', 'SECT_TP_NM': '-', 'TDD_CLSPRC': '4605', 'CMPPREVDD_PRC': '20', 'FLUC_RT': '0.44', 'TDD_OPNPRC': '4580', 'TDD_HGPRC': '4615', 'TDD_LWPRC': '4540', 'ACC_TRDVOL': '28863', 'ACC_TRDVAL': '132021245', 'MKTCAP': '208388955195', 'LIST_SHRS': '45252759'}
{'BAS_DD': '20241120', 'ISU_CD': '006840', 'ISU_NM': 'AK홀딩스', 'MKT_NM': 'KOSPI', 'SECT_TP_NM': '-', 'TDD_CLSPRC': '11360', 'CMPPREVDD_PRC': '160', 'FLUC_RT': '1.43', 'TDD_OPNPRC': '11230', 'TDD_HGPRC': '11380', 'TDD_LWPRC': '11180', 'ACC_TRDVOL': '1180', 'ACC_TRDVAL': '13267050', 'MKTCAP': '150492292960', 'LIST_SHRS': '13247561'}
{'BAS_DD': '20241120', 'ISU_CD': '027410', 'ISU_NM': 'BGF', 'MKT_NM': 'KOSPI', 'SECT_TP_NM': '-', 'TDD_CLSPRC': '3510', 'CMPPREVDD_PRC': '20', 'FLUC_RT': '0.57', 'TDD_OPNPRC': '3490', 'TDD_HGPRC': '3525', 'TDD_LWPRC': '3450', 'ACC_TRDVOL': '60793', 'ACC_TRDVAL':

In [1]:
import os
import requests
import mysql.connector
from datetime import datetime, timedelta

# 설정 정보
API_KEY = "A73F3BA4D1A04658BAAFE34134E80CE9E0FE4655"
BASE_URL = {
    'kospi': 'http://data-dbg.krx.co.kr/svc/apis/sto/stk_bydd_trd',
    'kosdaq': 'http://data-dbg.krx.co.kr/svc/apis/sto/ksq_bydd_trd',
    'konex': 'http://data-dbg.krx.co.kr/svc/apis/sto/knx_bydd_trd'
}
db_config = {
    'host': 'localhost',
    'user': 'pdc',
    'password': '1234',
    'database': 'CJD'
}
state_file = "/home/one/DC/progress_second.txt"

def read_last_date():
    """상태 파일에서 마지막으로 처리한 날짜를 읽습니다."""
    if os.path.exists(state_file):
        with open(state_file, "r") as file:
            return file.read().strip()
    return None

def write_last_date(date):
    """마지막으로 처리한 날짜를 상태 파일에 저장합니다."""
    with open(state_file, "w") as file:
        file.write(date)

def fetch_data(market, date):
    """API에서 데이터를 가져옵니다."""
    url = BASE_URL[market]
    headers = {'AUTH_KEY': API_KEY}
    params = {'basDd': date}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json().get('OutBlock_1', [])
        print(f"Fetched data for {market.upper()} on {date}: {len(data)} records")
        return data
    else:
        print(f"Error fetching data for {market.upper()} on {date}: {response.status_code}")
        return []

def save_to_mysql(data, table_name):
    """MySQL 데이터베이스에 데이터를 저장합니다."""
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        basDd TEXT,
        isuCd TEXT,
        isuNm TEXT,
        market TEXT,
        tddOpnprc TEXT,
        tddHgprc TEXT,
        tddLwprc TEXT,
        tddClsprc TEXT,
        accTrdvol TEXT,
        accTrdval TEXT
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    insert_query = f"""
    INSERT INTO {table_name} (
        basDd, isuCd, isuNm, market, tddOpnprc, tddHgprc, tddLwprc,
        tddClsprc, accTrdvol, accTrdval
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """
    
    # API 데이터 키와 데이터베이스 컬럼 이름 매핑
    field_mapping = {
        'BAS_DD': 'basDd',
        'ISU_CD': 'isuCd',
        'ISU_NM': 'isuNm',
        'MKT_NM': 'market',
        'TDD_OPNPRC': 'tddOpnprc',
        'TDD_HGPRC': 'tddHgprc',
        'TDD_LWPRC': 'tddLwprc',
        'TDD_CLSPRC': 'tddClsprc',
        'ACC_TRDVOL': 'accTrdvol',
        'ACC_TRDVAL': 'accTrdval'
    }
    
    for record in data:
        # 필드 매핑 및 빈 값 필터링
        mapped_record = {field_mapping[key]: record.get(key) for key in field_mapping if key in record}
        if all(value not in [None, ""] for value in mapped_record.values()):
            cursor.execute(insert_query, (
                mapped_record.get('basDd'), mapped_record.get('isuCd'), mapped_record.get('isuNm'),
                mapped_record.get('market'), mapped_record.get('tddOpnprc'), mapped_record.get('tddHgprc'),
                mapped_record.get('tddLwprc'), mapped_record.get('tddClsprc'), mapped_record.get('accTrdvol'),
                mapped_record.get('accTrdval')
            ))
    conn.commit()
    cursor.close()
    conn.close()
    print(f"Data saved to MySQL table {table_name}: {len(data)} records")

def collect_data():
    # 어제 날짜 계산
    yesterday_date = (datetime.today() - timedelta(days=1)).strftime('%Y%m%d')
    last_date = read_last_date()
    
    if last_date == yesterday_date:
        print("Data already processed for yesterday.")
        return
    
    for market in ['kospi', 'kosdaq', 'konex']:
        print(f"Fetching data for {market.upper()} on {yesterday_date}...")
        data = fetch_data(market, yesterday_date)
        if data:
            for record in data:
                record['market'] = market  # market 필드 추가
            save_to_mysql(data, "krx_market_data")
        else:
            print(f"No data fetched for {market.upper()} on {yesterday_date}.")
    
    write_last_date(yesterday_date)
    print("Yesterday's data processed and saved.")

if __name__ == "__main__":
    collect_data()


Fetching data for KOSPI on 20241120...
Fetched data for KOSPI on 20241120: 959 records
Data saved to MySQL table krx_market_data: 959 records
Fetching data for KOSDAQ on 20241120...
Fetched data for KOSDAQ on 20241120: 1776 records
Data saved to MySQL table krx_market_data: 1776 records
Fetching data for KONEX on 20241120...
Fetched data for KONEX on 20241120: 120 records
Data saved to MySQL table krx_market_data: 120 records
Yesterday's data processed and saved.


# 2

In [ ]:
import os
import requests
import mysql.connector
from datetime import datetime, timedelta
import logging
import subprocess

# 로깅 설정
LOG_FILE = "/home/one/DC/process.log"
logging.basicConfig(
    filename=LOG_FILE,
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

# API 키 입력
API_KEY = "A73F3BA4D1A04658BAAFE34134E80CE9E0FE4655"

# API 엔드포인트
BASE_URL = {
    'kospi': 'http://data-dbg.krx.co.kr/svc/apis/sto/stk_bydd_trd',
    'kosdaq': 'http://data-dbg.krx.co.kr/svc/apis/sto/ksq_bydd_trd',
    'konex': 'http://data-dbg.krx.co.kr/svc/apis/sto/knx_bydd_trd'
}

# MySQL 연결 설정
db_config = {
    'host': 'localhost',
    'user': 'pdc',
    'password': '1234',
    'database': 'CJD'
}

# 상태 파일 경로
state_file = "/home/one/DC/progress_state.txt"

# 요청 건수 제한 설정
MAX_REQUESTS = 10000  # 최대 요청 건수
current_requests = 0  # 현재 요청 건수


def validate_state_file():
    """상태 파일의 유효성을 확인하고 초기화합니다."""
    if not os.path.exists(state_file) or os.stat(state_file).st_size == 0:
        logging.warning("State file is missing or empty. Initializing with default date.")
        write_last_state(datetime.strptime("20100816", "%Y%m%d"), 0)
    else:
        try:
            with open(state_file, "r") as file:
                state = file.read().strip().split(",")
                datetime.strptime(state[0], "%Y%m%d")  # 날짜 검증
                if len(state) > 1:
                    int(state[1])  # offset 검증
        except (ValueError, IndexError):
            logging.error("State file is corrupted. Resetting to default state.")
            write_last_state(datetime.strptime("20100816", "%Y%m%d"), 0)


def read_last_state():
    """상태 파일에서 마지막으로 처리한 날짜와 offset을 읽습니다."""
    validate_state_file()  # 유효성 검사 추가
    with open(state_file, "r") as file:
        state = file.read().strip().split(",")
        last_date = datetime.strptime(state[0], "%Y%m%d")
        last_offset = int(state[1]) if len(state) > 1 else 0
        logging.info(f"Read last state: date={last_date}, offset={last_offset}")
        return last_date, last_offset


def write_last_state(date, offset):
    """마지막으로 처리한 날짜와 offset을 상태 파일에 저장합니다."""
    with open(state_file, "w") as file:
        file.write(f"{date.strftime('%Y%m%d')},{offset}")
    logging.info(f"Updated state file with date={date.strftime('%Y%m%d')} and offset={offset}")


# 데이터 수집 함수는 동일하지만, 각 주요 단계에서 상태 파일을 업데이트하여 오류 시 복구 가능
def fetch_data(market, date, offset=0):
    """API에서 데이터를 가져옵니다."""
    global current_requests
    if current_requests >= MAX_REQUESTS:
        logging.warning("Maximum API request limit reached. Stopping further requests.")
        return []
    
    url = BASE_URL[market]
    headers = {'AUTH_KEY': API_KEY}
    params = {'basDd': date, 'offset': offset}
    try:
        response = requests.get(url, headers=headers, params=params)
        current_requests += 1
        if response.status_code == 200:
            data = response.json().get('OutBlock_1', [])
            logging.info(f"Fetched {len(data)} records for {market.upper()} on {date} starting from offset {offset}")
            return data
        else:
            logging.error(f"Error fetching data for {market.upper()} on {date} at offset {offset}: {response.status_code}")
            return []
    except requests.exceptions.RequestException as e:
        logging.error(f"Request error for {market} on {date}: {e}")
        return []


def collect_data(current_date, last_offset, end_date, table_name):
    """데이터를 수집하고 MySQL에 저장."""
    global current_requests
    while current_date <= end_date:
        logging.info(f"Processing date: {current_date.strftime('%Y-%m-%d')}")
        for market in ['kospi', 'kosdaq', 'konex']:
            while True:
                if current_requests >= MAX_REQUESTS:
                    logging.warning("Reached API request limit. Stopping.")
                    write_last_state(current_date, last_offset)
                    return current_date
                
                data = fetch_data(market, current_date.strftime('%Y%m%d'), last_offset)
                if data:
                    # 저장 로직 생략
                    last_offset += len(data)
                    write_last_state(current_date, last_offset)
                else:
                    last_offset = 0
                    break
        current_date += timedelta(days=1)
        last_offset = 0
        write_last_state(current_date, last_offset)
    return current_date


def main():
    validate_state_file()
    current_date, last_offset = read_last_state()
    end_date = datetime.today() - timedelta(days=2)
    table_name = "krx_market_data"
    
    if current_date > end_date:
        logging.info("All data processed. Switching to next stage.")
        return
    
    collect_data(current_date, last_offset, end_date, table_name)


if __name__ == "__main__":
    main()
